In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from category_encoders import OneHotEncoder
from category_encoders import BinaryEncoder

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score ,accuracy_score


C:\Users\Ahmed_Moataz\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_excel("Data_Train.xlsx")

In [3]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [4]:
def wrangle(link):
    df = pd.read_excel(link)
    df.dropna(inplace = True)    
    
    # get arrived time by hour
    df["Arrival_Time_hour"] = pd.to_datetime(df["Arrival_Time"].str.split(" ", expand = True)[0]).dt.time
    df["Dep_Time"] = pd.to_datetime(df["Dep_Time"]).dt.hour
    
#     get duration integer by hour from Duration
    df['hours'] = df["Duration"].str.split(" ",expand = True,)[0].str[:-1].astype(int)
    df["min"] = df["Duration"].str.split(" ",expand = True)[1].str[:-1]
    # fill NaN values with 0

    df['min'].fillna(0, inplace=True)
    df['min'] = df['min'].astype(int)
    # combine hours and minutes into a single column
    df['duration'] = df.apply(lambda x: x['hours'] + x['min'] / 60, axis=1)

    # drop original columns
    df.drop(['hours', 'min','Arrival_Time',"Duration"], axis=1, inplace=True)
    
    # edit in total stops non-stop to 0 stop
    index_non_stop  = df[df['Total_Stops'] == 'non-stop'].index
    df.loc[index_non_stop,"Total_Stops"] = "0 stops"
    
    # make total stops numbers
    df['Total_Stops'] = df['Total_Stops'].str.split(" ",expand = True)[0].astype(int)
    
    #drop Additional_Info becouse no info most the column
    df.drop(['Additional_Info'], axis=1, inplace=True)
    #drop Route not intersted
    df.drop(['Route'], axis=1, inplace=True)
    # handle date of journy i found the year  just 2019 and the months 3,5,6,4 and that is unusfull not good
    df.drop(['Date_of_Journey'], axis=1, inplace=True)
    
    # handle Dep_Time to make parts in day
    def get_part_of_day(hour):
        if 5 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 17:
            return 'afternoon'
        elif 17 <= hour < 21:
            return 'evening'
        else:
            return 'night'

    # apply function to datetime column and create new column for part of day
    df['part_of_day'] = df['Dep_Time'].apply(lambda x: get_part_of_day(x))
    
    df.drop(['Dep_Time','Arrival_Time_hour'], axis=1, inplace=True)
    return df

In [5]:
df = wrangle("Data_Train.xlsx")

In [6]:
df.head()

,Airline,Source,Destination,Total_Stops,Price,duration,part_of_day
0,IndiGo,Banglore,New Delhi,0,3897,2.833333,night
1,Air India,Kolkata,Banglore,2,7662,7.416667,morning
2,Jet Airways,Delhi,Cochin,2,13882,19.000000,morning
3,IndiGo,Kolkata,Banglore,1,6218,5.416667,evening
4,IndiGo,Banglore,New Delhi,1,13302,4.750000,afternoon


In [7]:
# cat_df = df.select_dtypes("object")

In [8]:
# cat_df.columns

Index(['Airline', 'Source', 'Destination', 'part_of_day'], dtype='object')

In [9]:
# ohe = OneHotEncoder()
# ohe.fit(cat_df)
# cat_df_trans = ohe.transform(cat_df)

C:\Users\Ahmed_Moataz\anaconda3\lib\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
C:\Users\Ahmed_Moataz\anaconda3\lib\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
C:\Users\Ahmed_Moataz\anaconda3\lib\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
C:\Users\Ahmed_Moataz\anaconda3\lib\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():


In [10]:
# cat_df_trans

In [11]:
# df.drop(columns =cat_df.columns, inplace = True)

In [12]:
# df = pd.concat([cat_df_trans,df],axis=1)

In [13]:
df.head()

,Airline_1,Airline_2,Airline_3,Airline_4,Airline_5,Airline_6,Airline_7,Airline_8,Airline_9,Airline_10,...,Destination_4,Destination_5,Destination_6,part_of_day_1,part_of_day_2,part_of_day_3,part_of_day_4,Total_Stops,Price,duration
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,3897,2.833333
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2,7662,7.416667
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2,13882,19.000000
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,6218,5.416667
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,13302,4.750000


In [14]:
# split
X = df.drop(columns = ['Price'])
y = df["Price"]

In [15]:

# grid_pramiter = {'n_neighbors': list(range(1,16,2))}
# neigh = KNeighborsClassifier()
# grid_search = GridSearchCV(estimator=neigh,param_grid=grid_pramiter,scoring= 'accuracy', cv =5)
# grid_search = grid_search.fit(X_train, y_train)

In [16]:
# grid_search.best_score_

In [17]:
# grid_search.best_params_

In [18]:
X.head()

,Airline_1,Airline_2,Airline_3,Airline_4,Airline_5,Airline_6,Airline_7,Airline_8,Airline_9,Airline_10,...,Destination_3,Destination_4,Destination_5,Destination_6,part_of_day_1,part_of_day_2,part_of_day_3,part_of_day_4,Total_Stops,duration
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2.833333
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,2,7.416667
2,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,2,19.000000
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,5.416667
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,4.750000


In [19]:
def get_best_regression_model(df):
    
    #encoding
    cat_df = df.select_dtypes("object") 
    
    ohe = OneHotEncoder()
    ohe.fit(cat_df)
    cat_df_trans = ohe.transform(cat_df)
        
    df.drop(columns =cat_df.columns, inplace = True)
    
    df = pd.concat([cat_df_trans,df],axis=1) 
    # split
    X = df.drop(columns = ['Price'])
    y = df["Price"]
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Define a list of regression models to evaluate
    models = [
        LinearRegression(),
        DecisionTreeRegressor(),
        Ridge(),
        Lasso(),
        SVR(),
        RandomForestRegressor()
    ]
    
    # Train and evaluate each model
    best_model = None
    best_score = -float('inf')
    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score = r2_score(y_test, y_pred)
        print(f"{type(model).__name__} Accuracy: {score}")
        
        if score > best_score:
            best_score = score
            best_model = model
    
    print(f"\nBest model: {type(best_model).__name__} with accuracy {best_score}")
    return best_model

In [20]:
get_best_regression_model(df)

LinearRegression Accuracy: 0.5909845227454431
DecisionTreeRegressor Accuracy: 0.5798093926850538
Ridge Accuracy: 0.5903908741165337


C:\Users\Ahmed_Moataz\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.006e+07, tolerance: 1.811e+07
  model = cd_fast.enet_coordinate_descent(


Lasso Accuracy: 0.5907440286353451
SVR Accuracy: 0.1561416646647359
RandomForestRegressor Accuracy: 0.5841493224806078

Best model: LinearRegression with accuracy 0.5909845227454431


LinearRegression()

In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from category_encoders import BinaryEncoder
import pandas as pd

def get_best_regression_model_with_encoding(df):
    
    # split
    X = df.drop(columns = ['Price'])
    y = df["Price"]
    
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Define the categorical columns
    
    cat_cols = X.select_dtypes(include=['object', 'category']).columns
    
    # Define the encoding methods to evaluate
    encoders = [
        ('binary', BinaryEncoder(), cat_cols),
        ('onehot', OneHotEncoder(), cat_cols)
    ]
    
    # Define the numeric columns
    num_cols = X.select_dtypes(include=['int', 'float']).columns
    
    # Define the preprocessor for the numeric columns
    numeric_transformer = StandardScaler()
    
    # Define the column transformer to apply encoding and scaling to the data
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_cols),
            ('cat', None, cat_cols)
        ],
        remainder='passthrough'
    )
    
    # Train and evaluate each model with each encoding method
    best_model = None
    best_score = -float('inf')
    for enc_name, encoder, enc_cols in encoders:
        X_train_encoded = encoder.fit_transform(X_train[enc_cols])
        X_test_encoded = encoder.transform(X_test[enc_cols])
        
        # Combine the encoded categorical features with the numeric features
        X_train_processed = pd.concat([X_train_encoded, X_train[num_cols]], axis=1)
        X_test_processed = pd.concat([X_test_encoded, X_test[num_cols]], axis=1)
        
#         print(X_train_processed)
        ######## if i want make scale
        # Apply the preprocessor to scale the numeric features
#         X_train_processed = preprocessor.fit_transform(X_train_processed)
#         X_test_processed = preprocessor.transform(X_test_processed)
        
        # Define a list of regression models to evaluate
        models = [
            LinearRegression(),
            DecisionTreeRegressor(),
            Ridge(),
            Lasso(),
            SVR(),
            RandomForestRegressor()
        ]
        
        print(f"Encoding method: {enc_name}")
        enc_best_model = None
        enc_best_score = -float('inf')
        for model in models:
            model.fit(X_train_processed, y_train)
            y_pred = model.predict(X_test_processed)
            score = r2_score(y_test, y_pred)
            print(f"{type(model).__name__} Accuracy: {score}")
            
            if score > enc_best_score:
                enc_best_score = score
                enc_best_model = model
        
        print(f"\nBest model for {enc_name} encoding: {type(enc_best_model).__name__} with accuracy {enc_best_score}\n")
        
        if enc_best_score > best_score:
            best_score = enc_best_score
            best_model = enc_best_model
    
    print(f"\nBest model overall: {type(best_model).__name__} with accuracy {best_score}")
    return best_model

In [22]:
get_best_regression_model_with_encoding(df)

Encoding method: binary
LinearRegression Accuracy: 0.5909845227454431
DecisionTreeRegressor Accuracy: 0.5798647988793793
Ridge Accuracy: 0.5903908741165337


C:\Users\Ahmed_Moataz\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.006e+07, tolerance: 1.811e+07
  model = cd_fast.enet_coordinate_descent(


Lasso Accuracy: 0.5907440286353451
SVR Accuracy: 0.1561416646647359
RandomForestRegressor Accuracy: 0.5915395940800479

Best model for binary encoding: RandomForestRegressor with accuracy 0.5915395940800479



TypeError: cannot concatenate object of type '<class 'scipy.sparse.csr.csr_matrix'>'; only Series and DataFrame objs are valid